---
description: 使用 Langfuse 提示管理，在采用 OpenAI 函数调用时协作式地进行提示词的版本控制的实践手册。
category: 提示管理
---

# 示例：面向 OpenAI 函数调用的 Langfuse 提示管理（Python）

Langfuse 的[提示管理](https://langfuse.com/docs/prompts)可以在一个地方协作管理并进行提示词的版本控制。本文示例演示如何利用 Langfuse 提示的灵活 `config` 对象来保存函数调用选项和模型参数。

## 环境准备

In [ ]:
%pip install langfuse openai --upgrade

In [ ]:
import os

# 获取你的项目密钥
os.environ["LANGFUSE_PUBLIC_KEY"] = ""
os.environ["LANGFUSE_SECRET_KEY"] = ""
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

# OpenAI 密钥
os.environ["OPENAI_API_KEY"] = ""

In [14]:
from langfuse import get_client

langfuse = get_client()

True

## 在 Langfuse 提示管理中添加提示词

本示例通过 SDK 添加所用提示词。你也可以在 Langfuse 的 UI 中编辑并进行版本管理。

- `name`：在 Langfuse 提示管理中标识该提示词的名称
- 提示词中包含 `json_schema` 变量
- `config`：包含 `model_name`、`temperature` 与 `json_schema`
- `labels`：包含 `production`，以便立即作为默认版本使用

In [ ]:
langfuse.create_prompt(
    name="travel_planning",
    prompt="请基于用户的旅行需求生成结构化的旅行规划，并以 JSON 格式返回。使用如下 JSON 模式：{{json_schema}}",
    config={
        "model":"gpt-3.5-turbo-1106",
        "temperature": 0,
        "json_schema":{
            "destination": "string（目的地）",
            "start_date": "string（出发日期，YYYY-MM-DD）",
            "end_date": "string（返回日期，YYYY-MM-DD）",
            "duration_days": "number（行程天数）",
            "budget_level": "string（预算水平：经济/适中/高端）",
            "traveler_count": "number（出行人数）",
            "interests": "array of strings（兴趣偏好，如 美食/自然/博物馆/夜生活）",
            "must_try_foods": "array of strings（必吃清单）",
            "daily_itinerary": "array of strings（逐日行程概述）",
            "accommodation_suggestion": "string（住宿建议）",
            "transportation": "string（主要交通方式）",
            "notes": "string（注意事项）"
        }
    },
    labels=["production"]
);

在 Langfuse UI 中查看提示词

![Langfuse 提示管理](https://langfuse.com/images/docs/prompt-management-with-config-for-openai-functions.png)

## 示例应用

### 从 Langfuse 获取当前提示词版本

In [ ]:
prompt = langfuse.get_prompt("travel_planning")

现在可以使用该提示词来生成系统消息

In [17]:
prompt.compile(json_schema="TEST SCHEMA")

'Extract the key information from this text and return it in JSON format. Use the following schema: TEST SCHEMA'

其中还包含 `config` 对象

```python
prompt.config
```

```
{'model': 'gpt-3.5-turbo-1106',
 'json_schema': {
  'destination': 'string（目的地）',
  'start_date': 'string（出发日期，YYYY-MM-DD）',
  'end_date': 'string（返回日期，YYYY-MM-DD）',
  'duration_days': 'number（行程天数）',
  'budget_level': 'string（预算水平：经济/适中/高端）',
  'traveler_count': 'number（出行人数）',
  'interests': 'array of strings（兴趣偏好，如 美食/自然/博物馆/夜生活）',
  'must_try_foods': 'array of strings（必吃清单）',
  'daily_itinerary': 'array of strings（逐日行程概述）',
  'accommodation_suggestion': 'string（住宿建议）',
  'transportation': 'string（主要交通方式）',
  'notes': 'string（注意事项）'
 },
 'temperature': 0}
 ```

### 创建示例函数

在本示例中，我们通过从 `langfuse.openai` 导入，使用原生的 [Langfuse OpenAI 集成](https://langfuse.com/integrations/model-providers/openai-py)。这将启用 Langfuse 的[追踪](https://langfuse.com/docs/tracing)功能，但该功能并非使用 Langfuse 提示管理所必需。

In [19]:
from langfuse.openai import OpenAI
client = OpenAI()

使用 Langfuse 提示来构造 `plan_trip` 示例函数。

**注意：** 你可以在调用 `create` 方法时传入 `langfuse_prompt` 参数，将一次生成与对应的提示词版本关联到 Langfuse Tracing。更多关于与其他集成或 SDK 进行关联的方式，请参阅我们的[提示管理文档](https://langfuse.com/docs/prompts/get-started#link-with-langfuse-tracing-optional)。

In [ ]:
import json

def plan_trip(travel_request):
  # 将 JSON 模式转为字符串
  json_schema_str = ', '.join([f"'{key}': {value}" for key, value in prompt.config["json_schema"].items()])

  # 使用字符串化的 JSON 模式渲染系统提示词
  system_message = prompt.compile(json_schema=json_schema_str)

  # 组装为 OpenAI 消息格式
  messages = [
      {"role":"system","content": system_message},
      {"role":"user","content": travel_request}
  ]

  # 读取额外配置
  model = prompt.config["model"]
  temperature = prompt.config["temperature"]

  # 调用大模型接口
  res = client.chat.completions.create(
    model = model,
    temperature = temperature,
    messages = messages,
    response_format = { "type": "json_object" },
    langfuse_prompt = prompt # 在追踪中记录所用提示词版本
  )

  # 将响应解析为 JSON
  res = json.loads(res.choices[0].message.content)

  return res

### 运行示例

In [ ]:
# 旅行需求示例
TRAVEL_REQUEST = """
出发地：上海
目的地：东京
出发日期：2025-10-01
返回日期：2025-10-06
预算：适中
人数：2 人
兴趣偏好：美食、现代建筑、动漫文化
特殊需求：尽量避开人流高峰；每日步行不超过 12 公里
"""

In [ ]:
plan = plan_trip(TRAVEL_REQUEST)

```
{
 'destination': '东京',
 'start_date': '2025-10-01',
 'end_date': '2025-10-06',
 'duration_days': 6,
 'budget_level': '适中',
 'traveler_count': 2,
 'interests': ['美食', '现代建筑', '动漫文化'],
 'must_try_foods': ['寿司', '拉面', '天妇罗'],
 'daily_itinerary': [
   'D1: 抵达与休整；傍晚浅草寺与晴空塔夜景',
   'D2: 筑地周边美食与银座步行；teamLab Planets 夜场',
   'D3: 台场一日（高达、海边公园、购物）',
   'D4: 涩谷与新宿（街拍、都市建筑、高空观景）',
   'D5: 秋叶原与上野（动漫与博物馆）',
   'D6: 返程；按航班时间调整' 
 ],
 'accommodation_suggestion': '新宿/银座一带交通便利，便于多点放射游览',
 'transportation': '以地铁为主，配合步行与少量打车',
 'notes': '国庆期间部分景点人流较大，尽量选择早场/夜场；每日步行控制在 10-12 公里内，必要时增补地铁/公交。'
}
```

## 在 Langfuse 中查看追踪

由于我们使用了与 OpenAI SDK 的原生 Langfuse 集成，可以直接在 Langfuse 中查看此次调用的完整追踪。

![Langfuse 中 OpenAI 函数的追踪](https://langfuse.com/images/docs/openai-functions-trace-with-prompt-management.png)

## 在 Langfuse 中迭代提示词

现在可以在 Langfuse 的 UI 中直接迭代提示词，包括模型参数与函数调用选项，而无需修改代码或重新部署应用。